As this is the *N-th* iteration of this project, data files are currently all over the place. In this notebook, I hope to consolidate as much historical data that I have as possible and migrate it to a single MongoDB cluster.

# MongoDB Setup #

Now it's time to move these cleaned files into a MongoDB database

In [1]:
import os
import re
from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv()

True

In [2]:
#assigning environment variables

#MONGODB_URL inside .env file
MONGODB_URL = os.getenv("MONGODB_URL")
CLUSTER = os.getenv("CLUSTER")

In [3]:
#connect to MongoDB
client = MongoClient(MONGODB_URL)
db = client[CLUSTER]

In [4]:
type(db)

pymongo.database.Database

# Cleaning Column Names and Datatypes #

## Old School Bond ##

In [5]:
import pandas as pd
import numpy as np

In [6]:
#the largest old school bond data set
bond_df = pd.read_csv("hist_data/old_school_bond_appended.csv")
bond_df_copy = bond_df.copy()

In [7]:
bond_df.head()

,Index,Timestamps_close,Close,Timestamps_average,Average
0,0,2020/04/08,4134618,2020/04/08,4522980
1,1,2020/04/09,4009639,2020/04/09,4501601
2,2,2020/04/10,3903868,2020/04/10,4473252
3,3,2020/04/11,3853462,2020/04/11,4442300
4,4,2020/04/12,3941327,2020/04/12,4415261


**Problems**
- There are 2 timestamp columns
- There is a seccond Index column
- Timestamp column names are named strangely
- Timestamps are probably strings, not a date object

## Fixing Timestamps ##

In [8]:
#check that timestamp columns are identical
close_ts = bond_df["Timestamps_close"]
avg_ts = bond_df["Timestamps_average"]
truthy = close_ts == avg_ts
where = truthy[truthy==False]
where #these timestamps are certainly all the same and .:. redundant

Series([], dtype: bool)

In [9]:
#what's the data type for the timestamps?
type(bond_df["Timestamps_close"][0]) #convert to datetime objects

str

In [10]:
bond_df["Timestamps_close"] = pd.to_datetime(bond_df["Timestamps_close"])

In [11]:
type(bond_df["Timestamps_close"][9])

pandas._libs.tslibs.timestamps.Timestamp

In [12]:
bond_df

,Index,Timestamps_close,Close,Timestamps_average,Average
0,0,2020-04-08,4134618,2020/04/08,4522980
1,1,2020-04-09,4009639,2020/04/09,4501601
2,2,2020-04-10,3903868,2020/04/10,4473252
3,3,2020-04-11,3853462,2020/04/11,4442300
4,4,2020-04-12,3941327,2020/04/12,4415261
...,...,...,...,...,...
195,195,2020-10-20,5803452,2020/10/20,5424806
196,196,2020-10-21,5934796,2020/10/21,5451299
197,197,2020-10-22,5916807,2020/10/22,5475026
198,198,2020-10-23,5729726,2020/10/23,5490737


## Several Operations ##

- Remove the useless columns, Index and one of the timestamps columns
- Change the timestamps datatype from str to pd.Timestamp
- Rename the index
- Rename the columns (with units!)

In [13]:
#remove to index by date instead
bond_df = bond_df.drop("Index", axis=1)
#remove redundant timestamps
bond_df = bond_df.drop("Timestamps_average", axis=1)
#set the index to pd.Timestamp objects
bond_df = bond_df.set_index("Timestamps_close")
#rename the index 
bond_df.index.name = "Date"
#rename the columns
bond_df.columns = ["Close (GP)", "Average (GP)"]
bond_df

,Close (GP),Average (GP)
Date,,
2020-04-08,4134618,4522980
2020-04-09,4009639,4501601
2020-04-10,3903868,4473252
2020-04-11,3853462,4442300
2020-04-12,3941327,4415261
...,...,...
2020-10-20,5803452,5424806
2020-10-21,5934796,5451299
2020-10-22,5916807,5475026


In [14]:
bond_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 200 entries, 2020-04-08 to 2020-10-24
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Close (GP)    200 non-null    int64
 1   Average (GP)  200 non-null    int64
dtypes: int64(2)
memory usage: 4.7 KB


No missing values and datatypes are as expected

## Red Chinchompas ##

In [15]:
#the longest red chinchompas dataset as a csv
rchins = pd.read_csv("hist_data/red_chinchompa_appended.csv")
rchins_copy = rchins.copy()

In [16]:
rchins.tail() #ends on Oct. 24 2020

,Index,Item Timestamps,Timestamps_close,Close,Timestamps_average,Average,Timestamps,Volume
195,195,2020/10/20,2020/10/20,1182,2020/10/20,1105,2020/10/20,8191571.0
196,196,2020/10/21,2020/10/21,1168,2020/10/21,1103,2020/10/21,9429012.0
197,197,2020/10/22,2020/10/22,1155,2020/10/22,1101,2020/10/22,6979693.0
198,198,2020/10/23,2020/10/23,1145,2020/10/23,1099,2020/10/23,5877863.0
199,199,2020/10/24,2020/10/24,1137,2020/10/24,1098,2020/10/24,6976954.0


**Problems**
- There are 4 timestamp columns that I assume are all identical
- The second index column has a weird name "Unnamed:0"
- Volume is a decimal (probably float) but close and average look like ints

### Timestamps Problem ###

In [17]:
item_t = rchins["Item Timestamps"]
close_t = rchins["Timestamps_close"]
avg_t = rchins["Timestamps_average"]
t = rchins["Timestamps"]

from itertools import permutations
perm = permutations([item_t, close_t, avg_t, t], 2)
for i in list(perm):
    comparison_srs = i[0]==i[1]
    print(comparison_srs[comparison_srs==False])

Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)


### Dropping Redundant Timestamps ###

In [18]:
dropping = ["Item Timestamps", "Timestamps_close", "Timestamps_average"]
for col in dropping:
    rchins = rchins.drop(col, axis=1)
rchins.head()

,Index,Close,Average,Timestamps,Volume
0,0,1734,1762,2020/04/08,9968057.0
1,1,1741,1761,2020/04/09,5868933.0
2,2,1737,1759,2020/04/10,3882743.0
3,3,1726,1758,2020/04/11,11836445.0
4,4,1686,1756,2020/04/12,7549086.0


### Dropping the second column of indices ###

In [19]:
rchins = rchins.drop("Index", axis=1)
rchins.head()

,Close,Average,Timestamps,Volume
0,1734,1762,2020/04/08,9968057.0
1,1741,1761,2020/04/09,5868933.0
2,1737,1759,2020/04/10,3882743.0
3,1726,1758,2020/04/11,11836445.0
4,1686,1756,2020/04/12,7549086.0


### Fixing Time/Date Datatype ###

In [20]:
rchins["Timestamps"] = pd.to_datetime(rchins["Timestamps"])
type(rchins["Timestamps"][8])

pandas._libs.tslibs.timestamps.Timestamp

### Setting Index to Timestamps ###

In [21]:
rchins = rchins.set_index("Timestamps")
rchins

,Close,Average,Volume
Timestamps,,,
2020-04-08,1734,1762,9968057.0
2020-04-09,1741,1761,5868933.0
2020-04-10,1737,1759,3882743.0
2020-04-11,1726,1758,11836445.0
2020-04-12,1686,1756,7549086.0
...,...,...,...
2020-10-20,1182,1105,8191571.0
2020-10-21,1168,1103,9429012.0
2020-10-22,1155,1101,6979693.0


### Renaming Columns ###

In [22]:
rchins.index.name = "Date"
rchins.columns = ["Close (GP)", "Average (GP)", "Volume (Sold)"]
rchins

,Close (GP),Average (GP),Volume (Sold)
Date,,,
2020-04-08,1734,1762,9968057.0
2020-04-09,1741,1761,5868933.0
2020-04-10,1737,1759,3882743.0
2020-04-11,1726,1758,11836445.0
2020-04-12,1686,1756,7549086.0
...,...,...,...
2020-10-20,1182,1105,8191571.0
2020-10-21,1168,1103,9429012.0
2020-10-22,1155,1101,6979693.0


### Checking Column Datatypes ###

In [23]:
rchins.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 200 entries, 2020-04-08 to 2020-10-24
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Close (GP)     200 non-null    int64  
 1   Average (GP)   200 non-null    int64  
 2   Volume (Sold)  200 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 6.2 KB


Since units sold can't be fractional, let's set the datatype of volume to int

In [24]:
rchins["Volume (Sold)"] = rchins["Volume (Sold)"].astype("int")
rchins.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 200 entries, 2020-04-08 to 2020-10-24
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Close (GP)     200 non-null    int64
 1   Average (GP)   200 non-null    int64
 2   Volume (Sold)  200 non-null    int64
dtypes: int64(3)
memory usage: 6.2 KB


# Defining Helper Functions #

Since Old School Bonds do not have a volume column, there will be separate helper functions for Old School Bonds and the remaining items.

## Cleaning Functions ##

In [25]:
def clean_bond_data(bond_data):
    #data transformations
    
    #use the old Timestamps_close column for timestamps
    #convert datatype to pd.Timestamp
    bond_data["Timestamps_close"] = pd.to_datetime(bond_data["Timestamps_close"])
    
    #remove to index by date instead
    bond_data = bond_data.drop("Index", axis=1)
    #remove redundant timestamps columns
    bond_data = bond_data.drop("Timestamps_average", axis=1)
    
    #set the index to pd.Timestamp objects
    bond_data = bond_data.set_index("Timestamps_close")
    #rename the index 
    bond_data.index.name = "Date"
    #rename the columns
    bond_data.columns = ["Close (GP)", "Average (GP)"]

    return bond_data

In [26]:
def clean_item_data(item_data):
    #data transformations
    
    #drop many redundant timestamps columns
    dropping = ["Item Timestamps", "Timestamps_close", "Timestamps_average"]
    for col in dropping:
        item_data = item_data.drop(col, axis=1)
    
    #dropping the second indices column
    item_data = item_data.drop("Index", axis=1)
    
    #convert timestamps column to pd.Timestamp objects
    item_data["Timestamps"] = pd.to_datetime(item_data["Timestamps"])
    
    #index by Timestamps
    item_data = item_data.set_index("Timestamps")
    
    #renaming columns
    item_data.index.name = "Date"
    item_data.columns = ["Close (GP)", "Average (GP)", "Volume (Sold)"]
    
    #convert volume dtype to int
    item_data["Volume (Sold)"] = item_data["Volume (Sold)"].astype("int")
    
    return item_data

In [27]:
clean_bond_data(bond_df_copy)

,Close (GP),Average (GP)
Date,,
2020-04-08,4134618,4522980
2020-04-09,4009639,4501601
2020-04-10,3903868,4473252
2020-04-11,3853462,4442300
2020-04-12,3941327,4415261
...,...,...
2020-10-20,5803452,5424806
2020-10-21,5934796,5451299
2020-10-22,5916807,5475026


In [28]:
clean_item_data(rchins_copy)

,Close (GP),Average (GP),Volume (Sold)
Date,,,
2020-04-08,1734,1762,9968057
2020-04-09,1741,1761,5868933
2020-04-10,1737,1759,3882743
2020-04-11,1726,1758,11836445
2020-04-12,1686,1756,7549086
...,...,...,...
2020-10-20,1182,1105,8191571
2020-10-21,1168,1103,9429012
2020-10-22,1155,1101,6979693


# Dataset Investigation:  Will our cleaning work? #

In [29]:
#don't need to do this more than once
"""
for r, d, f in os.walk("hist_data/"):
    for file in f:
        if "appended" in file and "old" not in file:
            df = pd.read_csv(r+file)
            print(f"NAME:  {file}\nCOLUMNS: {df.columns}\n")
            name_prefix = re.sub("_appended.csv",".csv", file)
            print(f"PREFIX {name_prefix}")
            clean_df = clean_item_data(df)
            display(clean_df.head())
            display(clean_df.tail())
            clean_df.to_csv("clean_data/"+name_prefix)
"""

'\nfor r, d, f in os.walk("hist_data/"):\n    for file in f:\n        if "appended" in file and "old" not in file:\n            df = pd.read_csv(r+file)\n            print(f"NAME:  {file}\nCOLUMNS: {df.columns}\n")\n            name_prefix = re.sub("_appended.csv",".csv", file)\n            print(f"PREFIX {name_prefix}")\n            clean_df = clean_item_data(df)\n            display(clean_df.head())\n            display(clean_df.tail())\n            clean_df.to_csv("clean_data/"+name_prefix)\n'

In [30]:
#don't need to do this more than once
#old_school_csv = "hist_data/old_school_bond_appended.csv"
#clean_bond_data(pd.read_csv(old_school_csv)).to_csv("clean_data/old_school_bond.csv")

# Adding .csv to MongoDB #

MongoDB is already set up at the top of this notebook. I'll interact with the cluster via `db`.

Inside of this database `db` I need to make several `collections` (basically tables) for each of the datasets.

In the future, I'll do this for every tradeable. The historical data I'm working with in this notebook is a good start for my current playstyle. 

In [31]:
y = "yew_long_bow.csv"
print(y)
y = re.sub('_', ' ', y)
y = re.sub(".csv", '', y)
foo = [i.capitalize() for i in y.split()]
bar = ' '.join(foo)
bar

yew_long_bow.csv


'Yew Long Bow'

In [32]:
def make_collection_name(filename):
    n = re.sub('_', ' ', filename)
    n = re.sub('.csv', '', n)
    name = [i.capitalize() for i in n.split()]
    return ' '.join(name)

for r, d, f in os.walk("clean_data/"):
    for filename in f:
        print(make_collection_name(filename))

Old School Bond
Dragon Bones
Black Chinchompa
Magic Logs
Red Chinchompa
Yew Longbow
Black Dragonhide


In [33]:
import json

In [34]:
df = pd.read_csv("clean_data/black_chinchompa.csv")
df_json = json.loads(df.to_json(orient="records"))
df_json

[{'Date': '2020-04-08',
  'Close (GP)': 3403,
  'Average (GP)': 3524,
  'Volume (Sold)': 2934738},
 {'Date': '2020-04-09',
  'Close (GP)': 3379,
  'Average (GP)': 3517,
  'Volume (Sold)': 1970588},
 {'Date': '2020-04-10',
  'Close (GP)': 3364,
  'Average (GP)': 3510,
  'Volume (Sold)': 1329140},
 {'Date': '2020-04-11',
  'Close (GP)': 3374,
  'Average (GP)': 3504,
  'Volume (Sold)': 3775753},
 {'Date': '2020-04-12',
  'Close (GP)': 3400,
  'Average (GP)': 3495,
  'Volume (Sold)': 2212882},
 {'Date': '2020-04-13',
  'Close (GP)': 3407,
  'Average (GP)': 3487,
  'Volume (Sold)': 1568562},
 {'Date': '2020-04-14',
  'Close (GP)': 3395,
  'Average (GP)': 3479,
  'Volume (Sold)': 1796671},
 {'Date': '2020-04-15',
  'Close (GP)': 3336,
  'Average (GP)': 3467,
  'Volume (Sold)': 612489},
 {'Date': '2020-04-16',
  'Close (GP)': 3297,
  'Average (GP)': 3457,
  'Volume (Sold)': 1299774},
 {'Date': '2020-04-17',
  'Close (GP)': 3305,
  'Average (GP)': 3449,
  'Volume (Sold)': 2261863},
 {'Date': '

In [35]:
def fresh_populate_mongo_collection(db, collection_name, dataframe):
    """
    Remove any contents of a collection and populate it with data
    from the current dataframe
    
    db (pymongo.database.Database):  The database parent of a collection
    collection_name (str):  The desired name of the collection
    dataframe (pandas.DataFrame):  A dataframe holding the data you want to put into the collection
    """
    
    #convert csv to json; each dict is 1 document (aka row)
    data_json = json.loads(dataframe.to_json(orient="records"))
    
    #clear anything currently populating the collection
    db[collection_name].delete_many({ })
    
    #post the data to a collection
    db[collection_name].insert_many(data_json)
    
    return 

In [36]:
bond_collection_name = make_collection_name("old_school_bond.csv")
bond_clean = pd.read_csv("clean_data/old_school_bond.csv")
fresh_populate_mongo_collection(db, bond_collection_name, bond_clean)

In [37]:
for r, d, f in os.walk("clean_data/"):
    for filename in f:
        cn = make_collection_name(filename)
        clean_df = pd.read_csv("clean_data/"+filename)
        fresh_populate_mongo_collection(db, cn, clean_df)

Loading up MongoDB Compass shows that all of our data has been placed correctly!

# Quick Mongo Experiment #

In [38]:
cursor = db["Old School Bond"].find()
len(list(cursor))

200

So you retrieve every document in a collection by passing no arguments to `database.collection.find()`